In [3]:
from collections import Counter
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics 
%matplotlib inline 

/usr/local/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:
def read_rpkm_matrix(rpkm_file_path):
    # --------------------------------------
    # load count data and meta information
    # --------------------------------------
    n_header_lines = 1
    n_meta_fields = 4
    # 1. count number of genes
    rpkm_file = open(rpkm_file_path)
    num_genes = 0
    for (i, line) in enumerate(rpkm_file):
        if i == 0: # header line
            vals = line.rstrip().split('\t')
            num_features = len(vals)-n_meta_fields
        if i >= n_header_lines:
            num_genes += 1
    rpkm_file.close()
    print 'Reading: ' + rpkm_file_path
    print 'Number of genes: ' + str(num_genes)
    # 2. read the full rpkm matrix
    full_log_mtx = np.empty((num_genes, num_features))  # each row will be the feature profile for a given gene
    gene_info = ["NULL"]*num_genes
    print 'Reading:' + rpkm_file_path
    rpkm_file = open(rpkm_file_path)
    for (i, line) in enumerate(rpkm_file):
        vals = line.rstrip().split('\t')
        if i < n_header_lines: 
            first_fields = '\t'.join(vals[0:n_meta_fields]) # print header later
            tissues = vals[n_meta_fields:]
            continue
        idx = i - n_header_lines  # gene index 
        if (idx % 1000 == 0):
            print '    ' + str(idx) + ' genes read'
        gene_info[idx] = '\t'.join(vals[0:n_meta_fields])
        exp_levels = vals[n_meta_fields:]
        full_log_mtx[idx,:] = exp_levels 
    rpkm_file.close()
    print 'Completed reading full matrix. Dimensions: ' + str(full_log_mtx.shape)
    return full_log_mtx, gene_info, tissues


In [10]:
# load tissue median expression values from file
filename = '../data/small_example_data/small_log_median_transcript_rpkm_in_go_nonzero_exp.txt'
full_log_mtx, gene_info, tissues = read_rpkm_matrix(filename)
df = pd.DataFrame(data=full_log_mtx, index=gene_info, columns=tissues)
sorteddf = df.reindex_axis(sorted(df.columns), axis=1)
sorteddf.describe()


Reading: ../data/small_example_data/small_log_median_transcript_rpkm_in_go_nonzero_exp.txt
Number of genes: 199
Reading:../data/small_example_data/small_log_median_transcript_rpkm_in_go_nonzero_exp.txt
    0 genes read
Completed reading full matrix. Dimensions: (199, 53)


Adipose - Subcutaneous  Adipose - Visceral (Omentum)  Adrenal Gland  \
count              199.000000                    199.000000     199.000000   
mean                -0.004065                     -0.008642      -0.008743   
std                  0.982137                      0.978263       0.987154   
min                 -1.021948                     -1.027793      -0.998843   
25%                 -1.007446                     -1.002484      -0.979921   
50%                 -0.081543                     -0.119070      -0.200945   
75%                  0.622693                      0.604096       0.618457   
max                  3.272546                      3.309111       2.919825   

       Artery - Aorta  Artery - Coronary  Artery - Tibial     Bladder  \
count      199.000000         199.000000       199.000000  199.000000   
mean        -0.005728          -0.005563        -0.004178   -0.002958   
std          0.988666           0.982485         0.984251    0.983916   
min         -0.947267          -0.967345        -0.943476   -1.089491   
25%         -0.928772          -0.949064        -0.931296   -1.016186   
50%         -0.215681          -0.232929        -0.223748   -0.020571   
75%          0.630278           0.642996         0.655964    0.670932   
max          3.067857           3.300961         3.224132    3.186072   

       Brain - Amygdala  Brain - Anterior cingulate cortex (BA24)  \
count        199.000000                                199.000000   
mean          -0.008359                                 -0.005608   
std            0.979443                                  0.985164   
min           -0.950615                                 -0.971253   
25%           -0.911969                                 -0.935683   
50%           -0.296124                                 -0.302270   
75%            0.646692                                  0.658608   
max            3.085487                                  3.329483   

       Brain - Caudate (basal ganglia)     ...       \
count                       199.000000     ...        
mean                         -0.005080     ...        
std                           0.987389     ...        
min                          -0.958573     ...        
25%                          -0.940984     ...        
50%                          -0.232973     ...        
75%                           0.706516     ...        
max                           3.047687     ...        

       Skin - Not Sun Exposed (Suprapubic)  Skin - Sun Exposed (Lower leg)  \
count                           199.000000                      199.000000   
mean                             -0.012348                       -0.008834   
std                               0.980046                        0.980744   
min                              -1.029377                       -1.041906   
25%                              -0.995783                       -1.014995   
50%                              -0.122648                       -0.121372   
75%                               0.631095                        0.645321   
max                               3.557761                        3.379207   

       Small Intestine - Terminal Ileum      Spleen     Stomach      Testis  \
count                        199.000000  199.000000  199.000000  199.000000   
mean                          -0.009136   -0.007963   -0.012873   -0.006771   
std                            0.960415    0.993410    0.966461    0.990824   
min                           -1.131347   -1.002375   -0.996986   -1.087644   
25%                           -0.988137   -0.990597   -0.953317   -0.886936   
50%                           -0.061709   -0.115851   -0.133299   -0.239655   
75%                            0.537277    0.690815    0.531698    0.655433   
max                            2.975048    3.299823    3.229529    3.150713   

          Thyroid      Uterus      Vagina  Whole Blood  
count  199.000000  199.000000  199.000000   199.000000  

In [11]:
method = 'pearson' 
corrmat = sorteddf.corr(method)
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=1, square=True, ax=ax)
ax.set_title(method)

In [12]:
method = 'spearman' 
corrmat = sorteddf.corr(method)
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=1, square=True, ax=ax)
ax.set_title(method)
